# Exploratory Data Analysis

**Using Instacart's Public, Anonymized Dataset to predict whether an item will be in an Instacart user's next order**

## Table Summary

Here is my understanding of the data structure.

* `orders`
    * one row per order (index = order_id)
    * does not contain information about reorders
    * `eval_set` indicates whether the order is in the `train`/`test`/`prior`
        * the `test` set is data reserved for the testing of our final model
        * the `prior` and `train` eval_sets are defined below
    * columns:
        * `order_id`: order identifier
        * `user_id`: customer identifier
        * `eval_set`: which evaluation set this order belongs in (see `SET` described below)
        * `order_number`: the order sequence number for this user (1 = first, n = nth)
        * `order_dow`: the day of the week the order was placed on
        * `order_hour_of_day`: the hour of the day the order was placed on
        * `days_since_prior`: days since the last order, capped at 30 (with NAs for `order_number` = 1)

* `prior_orders`
    * information about orders prior to that users most recent order (~3.2M orders)
    * contains one row per item per order & whether or not each item is a 'reorder'
        * reorder: 1 if products has been ordered by this user in the past, 0 otherwise
    * columns:
        * `order_id`: foreign key
        * `product_id`: foreign key
        * `add_to_cart_order`: order in which each product was added to cart
        * `reordered`: 1 if this product has been ordered by this user in the past, 0 otherwise
        
    
* `train_orders`
    * training data supplied to participants of Kaggle competition
    * this table represents the users' most recent orders
    * contains one row per item per order & whether or not each item is a 'reorder'(for training data)
    * none of the rows in `train_orders` will be found in `prior_orders`
    * columns:
        * `order_id`: foreign key
        * `product_id`: foreign key
        * `add_to_cart_order`: order in which each product was added to cart
        * `reordered`: 1 if this product has been ordered by this user in the past, 0 otherwise
        
* `prod_detail`
    * this table is a combination of `products.csv`, `aisles.csv`, and `departments.csv`
        * created via SQL script (see `db_create.sql`)
    * contains all product details for each product

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import psycopg2


from db_config import get_db_params
from query_dfs import create_dfs

pd.set_option("display.max_columns", 101)

In [2]:
db_params = get_db_params()
conn = psycopg2.connect(**db_params)

### Predicting whether an item will be in an Instacart user's next order.

This is a classification problem. Our target variable is boolean (`reorder`). A `1` indicates the user did/will reorder a given product in their `train` order.

Let's first load in our data.

In [3]:
# read in pickled DFs generated by query_dfs.py

df_orders = pd.read_pickle("./pickle/df_orders.pickle")
df_train = pd.read_pickle("./pickle/df_train.pickle")
df_prior = pd.read_pickle("./pickle/df_prior.pickle")
df_prod_detail = pd.read_pickle("./pickle/df_prod_detail.pickle")

In [4]:
X = pd.read_pickle("./pickle/X_F.pickle")

In [5]:
X.head(3)

,product_id,user_id,user_total_prod_orders,cart,in_cart,last_cart,in_last_cart,qty_reordered,qty_sold,prod_reorder_pct,prod_prior_sales,prod_pct_reorders,prod_avg_atco,user_avg_cart_size,days_since_prior_order,order_hour_of_day,user_avg_spacing,streak_nan,up_buy_streak,up_n5_n_buys,up_n5_buy_ratio,up_atco_sum,up_atco_avg,prod_total_mkt_share,prod_total_mkt_share_log,aisle_total_sales,prod_aisle_mkt_share,prod_aisle_mkt_share_log,streak_abs
0,1,138,2,[42475],0,"[46802, 22128, 40199, 21573, 26152, 12341]",0,1136,1852,0.613391,1852,0.613391,5.801836,4.625,1.0,12,10.4,0,0,2.0,0.4,6,3.0,0.000057,-9.770711,234065,0.007912,-4.839333,2.0
1,907,138,2,[42475],0,"[46802, 22128, 40199, 21573, 26152, 12341]",0,1123,2025,0.554568,2025,0.554568,3.653333,4.625,1.0,12,10.4,1,0,0.0,0.0,5,2.5,0.000062,-9.681408,305655,0.006625,-5.016887,NaN
2,1000,138,1,[42475],0,"[46802, 22128, 40199, 21573, 26152, 12341]",0,1065,2610,0.408046,2610,0.408046,9.503448,4.625,1.0,12,10.4,1,0,0.0,0.0,5,5.0,0.000080,-9.427627,17368,0.150276,-1.895279,NaN


**Now let's take a look at the distribution of our target variable**

In [6]:
val_counts = X.in_cart.value_counts().to_list()
reords = val_counts[0]
non_reords = val_counts[1]
total_prods_ordered  = np.sum(val_counts)

print(f"""
    Total Products Ordered = {total_prods_ordered}
    Total Reorders = {reords} ({reords/total_prods_ordered*100:.2f}%)
    Total Non-Reorders = {non_reords} ({non_reords/total_prods_ordered*100:.2f}%)
    """)


    Total Products Ordered = 8474661
    Total Reorders = 7645837 (90.22%)
    Total Non-Reorders = 828824 (9.78%)
    


Alright...so we have a significant **class imbalance**. We'll take note of that, as it will be important when we start modeling our data later on.

Let's keep digging into our data to see if we can find anything exciting.

In [7]:
df_prior.head(3)

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,cart,in_cart
0,114,24954,1,0,91891,prior,1,0,11,NaN,"[24954, 1688, 37371, 5782, 1263, 23763, 24385,...",1
1,114,1688,2,0,91891,prior,1,0,11,NaN,"[24954, 1688, 37371, 5782, 1263, 23763, 24385,...",1
2,114,37371,3,0,91891,prior,1,0,11,NaN,"[24954, 1688, 37371, 5782, 1263, 23763, 24385,...",1


In [8]:
df_train.head(3)

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,6129,24852,1,1,38907,train,7,1,14,30.0
1,6129,48364,2,1,38907,train,7,1,14,30.0
2,6129,21903,3,1,38907,train,7,1,14,30.0


In [9]:
df_prod_detail.head(3)

,aisle_id,department_id,product_name,aisle,department
product_id,,,,,
17917,37,1,Pineapple Coconut & Cream Frozen Bars,ice cream ice,frozen
11468,42,1,Sausage & Veggie Quiche,frozen vegan vegetarian,frozen
22430,38,1,"Corn Dogs, Classic",frozen meals,frozen


Let's pull the top ten products in terms of reorders. Let's also filter out the products that aren't very popular so that one person reordering an obscure item repeatedly will not influence the outcome.

In [10]:
q = """
    SELECT product_detail.product_id, product_detail.product_name, product_detail.aisle_id, product_detail.aisle,
        product_detail.department_id, product_detail.department,
        SUM(reordered) AS qty_reordered,
        COUNT(order_id) AS qty_sold
        
    FROM prior_orders
    LEFT JOIN product_detail ON prior_orders.product_id = product_detail.product_id
    GROUP BY product_detail.product_id, product_detail.product_name,
        product_detail.aisle_id, product_detail.aisle,
        product_detail.department_id, product_detail.department
    HAVING COUNT(order_id) > 500
    ORDER BY qty_reordered DESC
    LIMIT 10
"""
#     
top_ten_reords = pd.read_sql_query(q, conn)
top_ten_reords['prod_reorder_pct'] = top_ten_reords.qty_reordered / top_ten_reords.qty_sold


In [11]:
top_ten_reords.index += 1
top_ten_reords

,product_id,product_name,aisle_id,aisle,department_id,department,qty_reordered,qty_sold,prod_reorder_pct
1,24852,Banana,24,fresh fruits,4,produce,398609,472565,0.843501
2,13176,Bag of Organic Bananas,24,fresh fruits,4,produce,315913,379450,0.832555
3,21137,Organic Strawberries,24,fresh fruits,4,produce,205845,264683,0.777704
4,21903,Organic Baby Spinach,123,packaged vegetables fruits,4,produce,186884,241921,0.772500
5,47209,Organic Hass Avocado,24,fresh fruits,4,produce,170131,213584,0.796553
6,47766,Organic Avocado,24,fresh fruits,4,produce,134044,176815,0.758103
7,27845,Organic Whole Milk,84,milk,16,dairy eggs,114510,137905,0.830354
8,47626,Large Lemon,24,fresh fruits,4,produce,106255,152657,0.696038
9,27966,Organic Raspberries,123,packaged vegetables fruits,4,produce,105409,137057,0.769089
10,16797,Strawberries,24,fresh fruits,4,produce,99802,142951,0.698155


Okay...lots of produce. 

Let's filter those out and see what we have next.

In [12]:
q = """
    SELECT product_detail.product_id, product_detail.product_name, product_detail.aisle_id, product_detail.aisle,
        product_detail.department_id, product_detail.department,
        SUM(reordered) AS qty_reordered,
        COUNT(order_id) AS qty_sold
        
    FROM prior_orders
    LEFT JOIN product_detail ON prior_orders.product_id = product_detail.product_id
    WHERE product_detail.department <> 'produce'
    GROUP BY product_detail.product_id, product_detail.product_name,
        product_detail.aisle_id, product_detail.aisle,
        product_detail.department_id, product_detail.department
    HAVING COUNT(order_id) > 500
    ORDER BY qty_reordered DESC
    LIMIT 10
"""
#     
top_ten_reords_no_produce = pd.read_sql_query(q, conn)
top_ten_reords_no_produce['prod_reorder_pct'] = top_ten_reords_no_produce.qty_reordered / top_ten_reords_no_produce.qty_sold


In [13]:
top_ten_reords_no_produce.index += 1
top_ten_reords_no_produce

,product_id,product_name,aisle_id,aisle,department_id,department,qty_reordered,qty_sold,prod_reorder_pct
1,27845,Organic Whole Milk,84,milk,16,dairy eggs,114510,137905,0.830354
2,49235,Organic Half & Half,53,cream,16,dairy eggs,59672,76360,0.781456
3,44632,Sparkling Water Grapefruit,115,water seltzer sparkling water,7,beverages,58513,75886,0.771064
4,27086,Half & Half,53,cream,16,dairy eggs,52728,69217,0.761778
5,30489,Original Hummus,67,fresh dips tapenades,20,deli,51690,71314,0.724823
6,22035,Organic Whole String Cheese,21,packaged cheese,16,dairy eggs,45639,59676,0.764780
7,19660,Spring Water,115,water seltzer sparkling water,7,beverages,45558,56087,0.812274
8,5077,100% Whole Wheat Bread,112,bread,3,bakery,44834,60816,0.737207
9,35951,Organic Unsweetened Almond Milk,91,soy lactosefree,16,dairy eggs,43875,57895,0.757837
10,5785,Organic Reduced Fat 2% Milk,84,milk,16,dairy eggs,38544,47839,0.805702


Alright, the above tables are sorted by total qty of reorders, and not `prod_reorder_pct`. Let's change that.

In [14]:
q = """
    SELECT product_id, product_name, aisle_id, aisle, department_id,
        department, qty_reordered, qty_sold, (qty_reordered / qty_sold) AS prod_reorder_pct
    
    FROM (

        SELECT product_detail.product_id, product_detail.product_name, product_detail.aisle_id, product_detail.aisle,
            product_detail.department_id, product_detail.department,
            SUM(reordered) AS qty_reordered,
            COUNT(order_id) AS qty_sold

        FROM prior_orders
        LEFT JOIN product_detail ON prior_orders.product_id = product_detail.product_id
        GROUP BY product_detail.product_id, product_detail.product_name,
            product_detail.aisle_id, product_detail.aisle,
            product_detail.department_id, product_detail.department
        HAVING COUNT(order_id) > 500
        ) AS subquery
    
    ORDER BY prod_reorder_pct
    LIMIT 10
"""
#     
top_ten_reord_pcts = pd.read_sql_query(q, conn)
top_ten_reord_pcts.index += 1
top_ten_reord_pcts


,product_id,product_name,aisle_id,aisle,department_id,department,qty_reordered,qty_sold,prod_reorder_pct
1,10,Sparkling Orange Juice & Prickly Pear Beverage,115,water seltzer sparkling water,7,beverages,1304,2572,0
2,23,Organic Turkey Burgers,49,packaged poultry,12,meat seafood,479,1068,0
3,25,Salted Caramel Lean Protein & Fiber Bar,3,energy granola bars,19,snacks,1391,2166,0
4,28,Wheat Chex Cereal,121,cereal,14,breakfast,369,706,0
5,32,Nacho Cheese White Bean Chips,107,chips pretzels,19,snacks,367,543,0
6,34,Peanut Butter Cereal,121,cereal,14,breakfast,3928,6536,0
7,37,Noodle Soup Mix With Chicken Broth,69,soup broth bouillon,15,canned goods,322,769,0
8,45,European Cucumber,83,fresh vegetables,4,produce,12069,18413,0
9,49,Vegetarian Grain Meat Sausages Italian - 4 CT,14,tofu meat alternatives,20,deli,1326,2308,0
10,1,Chocolate Sandwich Cookies,61,cookies cakes,19,snacks,1136,1852,0


In [15]:
len(X.order_id.unique)

AttributeError: 'DataFrame' object has no attribute 'order_id'